In [1]:
# Auto Reload
%load_ext autoreload
%autoreload 2

In [2]:
# Import Modules
import numpy as np
import copy
import pandas as pd
import json
import requests
import bs4
from bs4 import BeautifulSoup
from pprint import pprint

import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re
import os
import time

[nltk_data] Downloading package punkt to /Users/boogiedev/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/boogiedev/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# Import Custom Modules
from src.helpers import *

In [4]:
url = 'https://www.sephora.com/product/protini-tm-polypeptide-cream-P427421'

In [5]:
# chrome_options = Options()  
# # chrome_options.add_argument("--headless")
# driver = webdriver.Chrome(options=chrome_options)
# driver.get(url)
# driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
# time.sleep(2)
# driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
# time.sleep(2)

# review_click_six(driver)

# get_review_text(driver)
# page = driver.page_source

# driver.close()
# driver.quit()

In [6]:
# html = BeautifulSoup(page, 'html.parser')

In [7]:
# html.find

In [27]:
def review_click_six(driver:webdriver, n_reviews:int=100) -> None:
    '''Given a webdriver object, will click the "6 more reviews" button in order to load page'''
    start = 16
    temp = '/html/body/div[3]/div[5]/main/div[2]/div[2]/div/div[1]/div/div[%s]/button'
    for x in range(n_reviews//6):
        path = temp % str(start)
        start += 12
        try:
            driver.find_element_by_xpath(path).click()
            time.sleep(1)
        except:
            break
            
def get_review_text(driver:webdriver, n_reviews:int=100) -> str:
    '''Given a webdriver object, will all loaded reviews on page'''
    res = []
    start = 5
    temp = '/html/body/div[3]/div[5]/main/div[2]/div[2]/div/div[1]/div/div[%s]/div[1]/div[2]/div[1]'
    for i in range(n_reviews):
        path = temp % str(start)
        try:
            text = driver.find_element_by_xpath(path)
            print(f'Review #{i} Success')
        except:
            text = f'Review #{i} Error'
            print(text)
        try:
            res.append(text.text)
        except:
            res.append(text)
        start += 2
        
    return res
            

In [12]:
def get_n_reviews(item_url:str, n_reviews=100, verified=True, headless=False) -> str:
    '''Returns corpus of reviews created from scraping n_reviews reviews from product page for either verified or unverified reviews'''
    chrome_options = Options()
    if headless:
        chrome_options.add_argument("--headless")
        
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

    if verified:
        driver.find_element_by_xpath('/html/body/div[3]/div[5]/main/div[2]/div[2]/div/div[1]/div/div[3]/div/div/div[1]/div[1]/label/div[1]').click()
        time.sleep(2)
    
    review_click_six(driver, n_reviews)
    
    reviews = get_review_text(driver, n_reviews) 
    driver.close()
    driver.quit()
    
    return reviews

In [23]:
url = 'https://www.sephora.com/product/protini-tm-polypeptide-cream-P427421'
url = 'https://www.sephora.com/product/the-water-cream-P418218'
# test = get_n_reviews(url, 100)

In [24]:
# test

In [15]:
# Parse for item features
def get_static_meta(item_url:str) -> dict:
    '''Returns item/product features from static section of given sephora url'''
    # GET Request to URL
    page = requests.get(item_url)
    # Create BS4 Object
    html = BeautifulSoup(page.text, "html.parser")
    
    # Find Name and Brand
    brand, name = (x.get_text() for x in html.find("h1", attrs={"data-comp":True}))
    
    # Find n_loves (product likes)
    n_loves = html.find('span', attrs={"data-at":"product_love_count"}).get_text()
    
    # Find Star Rating
    stars = html.find("div", {'data-comp':"StarRating "})['aria-label']
    
    # Find Ingredients and Skin Type Target
    tabbed_block = html.find('div', {'data-at':'product_tabs_section'})
    
    # Ingredients
    ingredient_section = tabbed_block.findAll('div', {'aria-labelledby':True})[2]
    ingredients = ingredient_section.get_text(strip=False, separator=' |$%&*$| ').split(' |$%&*$| ')
    ingredients = parse_ingredients(ingredients)
    # Skin Type
    skin_section = tabbed_block.findAll('div', {'aria-labelledby':True})[0]
    skin_types = parse_skin_type(skin_section)
    
    
    return brand, name, n_loves, stars, ingredients, skin_types

In [16]:
def get_dynamic_meta(url:str, headless=False) -> tuple:
    '''Returns item/product features from scoll-loaded section of given sephora url'''
    # Init Chrome Driver Options
    chrome_options = Options()  
    # Run headless arg to avoid GUI
    if headless:
        chrome_options.add_argument("--headless")
    # Init Driver Instance -> load options
    driver = webdriver.Chrome(options=chrome_options)
    
    # Call url via driver
    driver.get(url)
    # Scroll down to load REVEIW section
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    page = driver.page_source
    
    # Create HTML object
    html = BeautifulSoup(page, 'html.parser')
    
    # Find n_reviews 
    n_reviews = html.find("div", {'data-comp':"ReviewsStats Box "}).find('span').get_text()
    
    # Find Star Distribution
    star_info = html.find('table', {'data-comp':'HistogramChart '})
    star_counts = star_info.get_text(separator=' ').split()[2::3]
    star_dist = [f'{num}_stars: {star_counts[i]}' for num, i in zip(range(5 ,0, -1), range(len(star_counts)))]
    
    driver.close()
    driver.quit()
    
    return n_reviews, star_dist

In [17]:
def parse_ingredients(text_lists:list) -> list:
    '''Returns list of ingredients from a given text block, runs though pipeline to ensure only ingredients are taken'''
    stop_set = set(stopwords.words('english'))
    
    backup = ''
    for text in text_lists:
        set_i = set(text.lower().split())
        if text:
            if text.strip()[0] != '-':
                return text
            elif not (stop_set.intersection(set_i)):
                backup = text
    return backup


In [18]:
def parse_skin_type(text_block:bs4.element) -> str:
    '''Returns list of skin types from a given text block, runs though pipeline to ensure only valid types are taken'''
    html = text_block
    
    # Extract all break tags from text_block
    [e.extract() for e in html.findAll('br')]
    
    route = html.find('b', text=re.compile("skin", re.IGNORECASE))
    res = ''
    
    item = route.next_sibling
    while isinstance(item, bs4.element.NavigableString):
        res += item
        item = item.next_sibling
    return res
    
    
# parse_skin_type(skin_type)

In [29]:
def sephora_scrape(url:str, n_reviews=100, verified=True, headless=False) -> tuple:
    '''Returns dictionary of needed features parsed from sephora.com link using selenium for dynamic page loading'''
    # Statically scrape product meta info
    brand, name, n_loves, stars, ingredients, skin_types = get_static_meta(url) 
    
    # Dynamically scrape product review section
    num_reviews, star_dist = get_dynamic_meta(url)
    
    # Get item corpus (review text)
    reviews = get_n_reviews(url, n_reviews=n_reviews, verified=verified, headless=headless)
    
    return brand, name, n_loves, stars, ingredients, skin_types, num_reviews, star_dist, reviews

In [30]:
info = sephora_scrape(url='https://www.sephora.com/product/the-water-cream-P418218', n_reviews=50, verified=False)

Review #0 Success
Review #1 Success
Review #2 Success
Review #3 Success
Review #4 Success
Review #5 Success
Review #6 Success
Review #7 Success
Review #8 Success
Review #9 Success
Review #10 Success
Review #11 Success
Review #12 Success
Review #13 Success
Review #14 Success
Review #15 Success
Review #16 Success
Review #17 Success
Review #18 Success
Review #19 Success
Review #20 Success
Review #21 Success
Review #22 Success
Review #23 Success
Review #24 Success
Review #25 Success
Review #26 Success
Review #27 Success
Review #28 Success
Review #29 Success
Review #30 Success
Review #31 Success
Review #32 Success
Review #33 Success
Review #34 Success
Review #35 Success
Review #36 Success
Review #37 Success
Review #38 Success
Review #39 Success
Review #40 Success
Review #41 Success
Review #42 Success
Review #43 Success
Review #44 Success
Review #45 Success
Review #46 Success
Review #47 Success
Review #48 Success
Review #49 Success


In [31]:
info

('Tatcha',
 'The Water Cream',
 '232981',
 '4 stars',
 'Water, Saccharomyces/Camellia Sinensis Leaf/Cladosiphon Okamuranus/Rice Ferment Filtrate*, Dimethicone, Propanediol, Glycerin, Diglycerin, Diphenylsiloxy Phenyl Trimethicone, Gold, Belamcanda Chinensis Root Extract, Rosa Multiflora Fruit Extract, Houttuynia Cordata Extract, Sophora Angustifolia Root Extract, Sodium Hyaluronate, Lecithin, Pistacia Lentiscus (Mastic) Gum, Sodium Chloride, Sodium Citrate, Mica, Peg-9 Polydimethylsiloxyethyl Dimethicone, Dimethicone/Peg-10/15 Crosspolymer, Dimethicone/Phenyl Vinyl Dimethicone Crosspolymer, Disodium Edta, Tin Oxide, Titanium Dioxide, Butylene Glycol, Ethylhexylglycerin, Parfum/Fragrance, Alcohol, Phenoxyethanol. *Hadasei-3.\r\n',
 '✔ Normal✔ Oily✔ Combination✔  Sensitive\r\n',
 '3028 reviews',
 ['5_stars: 1816',
  '4_stars: 392',
  '3_stars: 307',
  '2_stars: 231',
  '1_stars: 282'],
 ["Verified Purchase\n14 h ago\nQuality product!\nThis cream is a dream. Light weight and has a non gre

In [33]:
sephora_scrape('https://www.sephora.com/product/the-true-cream-aqua-bomb-P394639?icid2=products%20grid:p394639', verified=False)

Review #0 Success
Review #1 Success
Review #2 Success
Review #3 Success
Review #4 Success
Review #5 Success
Review #6 Success
Review #7 Success
Review #8 Success
Review #9 Success
Review #10 Success
Review #11 Success
Review #12 Success
Review #13 Success
Review #14 Success
Review #15 Success
Review #16 Success
Review #17 Success
Review #18 Success
Review #19 Success
Review #20 Success
Review #21 Success
Review #22 Success
Review #23 Success
Review #24 Success
Review #25 Success
Review #26 Success
Review #27 Success
Review #28 Success
Review #29 Success
Review #30 Success
Review #31 Success
Review #32 Success
Review #33 Success
Review #34 Success
Review #35 Success
Review #36 Success
Review #37 Success
Review #38 Success
Review #39 Success
Review #40 Success
Review #41 Success
Review #42 Success
Review #43 Success
Review #44 Success
Review #45 Success
Review #46 Success
Review #47 Success
Review #48 Success
Review #49 Success
Review #50 Success
Review #51 Success
Review #52 Success
Rev

('belif',
 'The True Cream Aqua Bomb',
 '190974',
 '4.5 stars',
 'Water, Dipropylene Glycol, Glycerin, Methl Trimethicone, Alcohol Denat, Dimethicone, Cyclopentasiloxane, 1,2-Hexanediol, Malakite Extract, Caprylic/Capric Triglyceride, Pentaerythrityl Tetraethylhexanoate, PEG/PPG/Polybutylene Glycol-8/5/3 Glycerin, Alchemilla Vulgaris Leaf Extract*, Equisetum Arvense Leaf Extract*, Stellaria Media (Chickweed) Extract*, Urtica Dioica (Nettle) Leaf Extract*, Plantago Lanceolata Leaf Extract*, Avena Sativa (Oat) Kernel Extract**, Calendula Officinalis Flower Extract**, Nepeta Cataria Extract**, Rubus Idaeus (Raspberry) Leaf Extract**, Baptisia Tinctoria Root Extract**, Dimethiconol, Polymethylsilsesquioxane, Sodium Acrylate/Acryloyldimethyltaurate/Dimethylacrylamide Crosspolymer, Isohexadecane, Polysorbate 60, Ceramide 3, Cholesterol, Butyrospermum Parkii (Shea) Butter, Phenl Trimethicone, Pentaerythrityl Tetraisostearate, Panthenol, Squalane, Triethylhexanoin, Macadamia Ternifolia Seed Oi